In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import rho_plus as rp

theme, cs = rp.mpl_setup(False)

In [7]:
import re, numpy as np, os, sys, pandas
from pymatgen.core import Composition
from matminer.featurizers.base import MultipleFeaturizer
from matminer.featurizers import composition as cf
from matminer.featurizers.conversions import StrToComposition
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from matminer.datasets import load_dataset

df = load_dataset('mp_nostruct_20181018')
df

/home/nicholas/anaconda3/envs/slices/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,mpid,formula,e_hull,gap pbe,mu_b,elastic anisotropy,bulk modulus,shear modulus,e_form
0,mp-85,In,0.003319,0.0000,2.700000e-05,1.044699,33.154748,4.904836,0.003319
1,mp-110,Mg,0.039182,0.0000,-1.360000e-05,-11.326659,35.636106,1.830272,0.039182
2,mp-20,Be,0.108143,0.0000,1.000000e-07,8.030000,124.000000,84.000000,0.108143
3,mp-8640,Hf,0.071216,0.0000,-2.050000e-05,0.881277,101.242732,44.836516,0.071216
4,mp-674158,P,3.509988,2.0113,3.000042e+00,10.884643,0.327165,-0.064038,3.509988
...,...,...,...,...,...,...,...,...,...
83984,mp-4446,Sr3(GaO3)2,0.000691,3.5262,0.000000e+00,NaN,NaN,NaN,-2.832238
83985,mp-3393,Sr3Al2O6,0.000000,4.2046,0.000000e+00,NaN,NaN,NaN,-3.358646
83986,mp-24696,MgSb2(H4O3)6,0.028109,3.2827,-4.338000e-04,NaN,NaN,NaN,-1.533338
83987,mp-23984,GaH18C3(N3F2)3,0.000000,4.9759,2.460000e-05,NaN,NaN,NaN,-1.066094


In [8]:
df.dropna(subset=['formula'], inplace=True)
df['comp'] = [Composition(x) for x in df['formula']]
df.head()

,mpid,formula,e_hull,gap pbe,mu_b,elastic anisotropy,bulk modulus,shear modulus,e_form,comp
0,mp-85,In,0.003319,0.0000,2.700000e-05,1.044699,33.154748,4.904836,0.003319,(In)
1,mp-110,Mg,0.039182,0.0000,-1.360000e-05,-11.326659,35.636106,1.830272,0.039182,(Mg)
2,mp-20,Be,0.108143,0.0000,1.000000e-07,8.030000,124.000000,84.000000,0.108143,(Be)
3,mp-8640,Hf,0.071216,0.0000,-2.050000e-05,0.881277,101.242732,44.836516,0.071216,(Hf)
4,mp-674158,P,3.509988,2.0113,3.000042e+00,10.884643,0.327165,-0.064038,3.509988,(P)


In [9]:
df['num_atoms'] = [c.num_atoms for c in df['comp']]
df

,mpid,formula,e_hull,gap pbe,mu_b,elastic anisotropy,bulk modulus,shear modulus,e_form,comp,num_atoms
0,mp-85,In,0.003319,0.0000,2.700000e-05,1.044699,33.154748,4.904836,0.003319,(In),1.0
1,mp-110,Mg,0.039182,0.0000,-1.360000e-05,-11.326659,35.636106,1.830272,0.039182,(Mg),1.0
2,mp-20,Be,0.108143,0.0000,1.000000e-07,8.030000,124.000000,84.000000,0.108143,(Be),1.0
3,mp-8640,Hf,0.071216,0.0000,-2.050000e-05,0.881277,101.242732,44.836516,0.071216,(Hf),1.0
4,mp-674158,P,3.509988,2.0113,3.000042e+00,10.884643,0.327165,-0.064038,3.509988,(P),1.0
...,...,...,...,...,...,...,...,...,...,...,...
83984,mp-4446,Sr3(GaO3)2,0.000691,3.5262,0.000000e+00,NaN,NaN,NaN,-2.832238,"(Sr, Ga, O)",11.0
83985,mp-3393,Sr3Al2O6,0.000000,4.2046,0.000000e+00,NaN,NaN,NaN,-3.358646,"(Sr, Al, O)",11.0
83986,mp-24696,MgSb2(H4O3)6,0.028109,3.2827,-4.338000e-04,NaN,NaN,NaN,-1.533338,"(Mg, Sb, H, O)",45.0
83987,mp-23984,GaH18C3(N3F2)3,0.000000,4.9759,2.460000e-05,NaN,NaN,NaN,-1.066094,"(Ga, H, C, N, F)",37.0


In [10]:
df.eval('''
delta_e = e_form
magmom_pa = mu_b / num_atoms
bandgap = `gap pbe`
''', inplace=True)
df

,mpid,formula,e_hull,gap pbe,mu_b,elastic anisotropy,bulk modulus,shear modulus,e_form,comp,num_atoms,delta_e,magmom_pa,bandgap
0,mp-85,In,0.003319,0.0000,2.700000e-05,1.044699,33.154748,4.904836,0.003319,(In),1.0,0.003319,2.700000e-05,0.0000
1,mp-110,Mg,0.039182,0.0000,-1.360000e-05,-11.326659,35.636106,1.830272,0.039182,(Mg),1.0,0.039182,-1.360000e-05,0.0000
2,mp-20,Be,0.108143,0.0000,1.000000e-07,8.030000,124.000000,84.000000,0.108143,(Be),1.0,0.108143,1.000000e-07,0.0000
3,mp-8640,Hf,0.071216,0.0000,-2.050000e-05,0.881277,101.242732,44.836516,0.071216,(Hf),1.0,0.071216,-2.050000e-05,0.0000
4,mp-674158,P,3.509988,2.0113,3.000042e+00,10.884643,0.327165,-0.064038,3.509988,(P),1.0,3.509988,3.000042e+00,2.0113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83984,mp-4446,Sr3(GaO3)2,0.000691,3.5262,0.000000e+00,NaN,NaN,NaN,-2.832238,"(Sr, Ga, O)",11.0,-2.832238,0.000000e+00,3.5262
83985,mp-3393,Sr3Al2O6,0.000000,4.2046,0.000000e+00,NaN,NaN,NaN,-3.358646,"(Sr, Al, O)",11.0,-3.358646,0.000000e+00,4.2046
83986,mp-24696,MgSb2(H4O3)6,0.028109,3.2827,-4.338000e-04,NaN,NaN,NaN,-1.533338,"(Mg, Sb, H, O)",45.0,-1.533338,-9.640000e-06,3.2827
83987,mp-23984,GaH18C3(N3F2)3,0.000000,4.9759,2.460000e-05,NaN,NaN,NaN,-1.066094,"(Ga, H, C, N, F)",37.0,-1.066094,6.648649e-07,4.9759


In [11]:
df.to_csv('mpc_data.csv')

In [12]:
feature_calculators = MultipleFeaturizer([
    cf.Stoichiometry(),
    cf.ElementProperty.from_preset("magpie"),
    cf.ValenceOrbital(props=['avg']),
    cf.IonProperty(fast=True)])

In [13]:
X = feature_calculators.featurize_dataframe(df[['comp']], col_id='comp')
X

MultipleFeaturizer:   4%|▍         | 3500/83987 [00:00<00:06, 12775.49it/s]/home/nicholas/anaconda3/envs/slices/lib/python3.9/site-packages/pymatgen/core/periodic_table.py:120: UserWarning: No Pauling electronegativity for Ar. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(
/home/nicholas/anaconda3/envs/slices/lib/python3.9/site-packages/pymatgen/core/periodic_table.py:120: UserWarning: No Pauling electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(
/home/nicholas/anaconda3/envs/slices/lib/python3.9/site-packages/pymatgen/core/periodic_table.py:120: UserWarning: No Pauling electronegativity for Ne. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(
MultipleFeaturizer:  50%|█████     | 42000/83987 [00:05<00:03

,comp,0-norm,2-norm,3-norm,5-norm,7-norm,10-norm,MagpieData minimum Number,MagpieData maximum Number,MagpieData range Number,MagpieData mean Number,MagpieData avg_dev Number,MagpieData mode Number,MagpieData minimum MendeleevNumber,MagpieData maximum MendeleevNumber,MagpieData range MendeleevNumber,MagpieData mean MendeleevNumber,MagpieData avg_dev MendeleevNumber,MagpieData mode MendeleevNumber,MagpieData minimum AtomicWeight,MagpieData maximum AtomicWeight,MagpieData range AtomicWeight,MagpieData mean AtomicWeight,MagpieData avg_dev AtomicWeight,MagpieData mode AtomicWeight,MagpieData minimum MeltingT,MagpieData maximum MeltingT,MagpieData range MeltingT,MagpieData mean MeltingT,MagpieData avg_dev MeltingT,MagpieData mode MeltingT,MagpieData minimum Column,MagpieData maximum Column,MagpieData range Column,MagpieData mean Column,MagpieData avg_dev Column,MagpieData mode Column,MagpieData minimum Row,MagpieData maximum Row,MagpieData range Row,...,MagpieData mean NfUnfilled,MagpieData avg_dev NfUnfilled,MagpieData mode NfUnfilled,MagpieData minimum NUnfilled,MagpieData maximum NUnfilled,MagpieData range NUnfilled,MagpieData mean NUnfilled,MagpieData avg_dev NUnfilled,MagpieData mode NUnfilled,MagpieData minimum GSvolume_pa,MagpieData maximum GSvolume_pa,MagpieData range GSvolume_pa,MagpieData mean GSvolume_pa,MagpieData avg_dev GSvolume_pa,MagpieData mode GSvolume_pa,MagpieData minimum GSbandgap,MagpieData maximum GSbandgap,MagpieData range GSbandgap,MagpieData mean GSbandgap,MagpieData avg_dev GSbandgap,MagpieData mode GSbandgap,MagpieData minimum GSmagmom,MagpieData maximum GSmagmom,MagpieData range GSmagmom,MagpieData mean GSmagmom,MagpieData avg_dev GSmagmom,MagpieData mode GSmagmom,MagpieData minimum SpaceGroupNumber,MagpieData maximum SpaceGroupNumber,MagpieData range SpaceGroupNumber,MagpieData mean SpaceGroupNumber,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber,avg s valence electrons,avg p valence electrons,avg d valence electrons,avg f valence electrons,compound possible,max ionic char,avg ionic char
0,(In),1,1.000000,1.000000,1.000000,1.000000,1.000000,49.0,49.0,0.0,49.000000,0.000000,49.0,75.0,75.0,0.0,75.000000,0.000000,75.0,114.818000,114.818000,0.000000,114.818000,0.000000,114.818000,429.75,429.75,0.00,429.750000,0.000000,429.75,13.0,13.0,0.0,13.000000,0.000000,13.0,5.0,5.0,0.0,...,0.0,0.0,0.0,5.0,5.0,0.0,5.000000,0.000000,5.0,24.260000,24.260000,0.0000,24.260000,0.000000,24.260000,0.000,0.000,0.000,0.000000,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,139.0,139.0,0.0,139.000000,0.000000,139.0,2.000000,1.000000,10.000000,0.0,True,0.000000,0.000000
1,(Mg),1,1.000000,1.000000,1.000000,1.000000,1.000000,12.0,12.0,0.0,12.000000,0.000000,12.0,68.0,68.0,0.0,68.000000,0.000000,68.0,24.305000,24.305000,0.000000,24.305000,0.000000,24.305000,923.00,923.00,0.00,923.000000,0.000000,923.00,2.0,2.0,0.0,2.000000,0.000000,2.0,3.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,22.890000,22.890000,0.0000,22.890000,0.000000,22.890000,0.000,0.000,0.000,0.000000,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,194.0,194.0,0.0,194.000000,0.000000,194.0,2.000000,0.000000,0.000000,0.0,True,0.000000,0.000000
2,(Be),1,1.000000,1.000000,1.000000,1.000000,1.000000,4.0,4.0,0.0,4.000000,0.000000,4.0,67.0,67.0,0.0,67.000000,0.000000,67.0,9.012182,9.012182,0.000000,9.012182,0.000000,9.012182,1560.00,1560.00,0.00,1560.000000,0.000000,1560.00,2.0,2.0,0.0,2.000000,0.000000,2.0,2.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,7.890000,7.890000,0.0000,7.890000,0.000000,7.890000,0.000,0.000,0.000,0.000000,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,194.0,194.0,0.0,194.000000,0.000000,194.0,2.000000,0.000000,0.000000,0.0,True,0.000000,0.000000
3,(Hf),1,1.000000,1.000000,1.000000,1.000000,1.000000,72.0,72.0,0.0,72.000000,0.000000,72.0,45.0,45.0,0.0,45.000000,0.000000,45.0,178.490000,178.490000,0.000000,178.490000,0.000000,178.490000,2506.00,2506.00,0.00,2506.000000,0.000000,2506.00,4.0,4.0,0.0,4.000000,0.000000,4.0,6.0,6.0,0.0

In [14]:
# magmom_pa
# energy_pa
# stability
# bandgap
# volume_pa

X['magmom_pa'] = df['magmom_pa']
# X['energy_pa'] = np.nan
# X['stability'] = np.nan
X['bandgap'] = df['bandgap']
# X['volume_pa'] = np.nan
X['delta_e'] = df['delta_e']
X.shape

(83987, 149)

In [15]:
X['comp'] = [c.to_pretty_string() for c in X['comp']]
X

,comp,0-norm,2-norm,3-norm,5-norm,7-norm,10-norm,MagpieData minimum Number,MagpieData maximum Number,MagpieData range Number,MagpieData mean Number,MagpieData avg_dev Number,MagpieData mode Number,MagpieData minimum MendeleevNumber,MagpieData maximum MendeleevNumber,MagpieData range MendeleevNumber,MagpieData mean MendeleevNumber,MagpieData avg_dev MendeleevNumber,MagpieData mode MendeleevNumber,MagpieData minimum AtomicWeight,MagpieData maximum AtomicWeight,MagpieData range AtomicWeight,MagpieData mean AtomicWeight,MagpieData avg_dev AtomicWeight,MagpieData mode AtomicWeight,MagpieData minimum MeltingT,MagpieData maximum MeltingT,MagpieData range MeltingT,MagpieData mean MeltingT,MagpieData avg_dev MeltingT,MagpieData mode MeltingT,MagpieData minimum Column,MagpieData maximum Column,MagpieData range Column,MagpieData mean Column,MagpieData avg_dev Column,MagpieData mode Column,MagpieData minimum Row,MagpieData maximum Row,MagpieData range Row,...,MagpieData minimum NUnfilled,MagpieData maximum NUnfilled,MagpieData range NUnfilled,MagpieData mean NUnfilled,MagpieData avg_dev NUnfilled,MagpieData mode NUnfilled,MagpieData minimum GSvolume_pa,MagpieData maximum GSvolume_pa,MagpieData range GSvolume_pa,MagpieData mean GSvolume_pa,MagpieData avg_dev GSvolume_pa,MagpieData mode GSvolume_pa,MagpieData minimum GSbandgap,MagpieData maximum GSbandgap,MagpieData range GSbandgap,MagpieData mean GSbandgap,MagpieData avg_dev GSbandgap,MagpieData mode GSbandgap,MagpieData minimum GSmagmom,MagpieData maximum GSmagmom,MagpieData range GSmagmom,MagpieData mean GSmagmom,MagpieData avg_dev GSmagmom,MagpieData mode GSmagmom,MagpieData minimum SpaceGroupNumber,MagpieData maximum SpaceGroupNumber,MagpieData range SpaceGroupNumber,MagpieData mean SpaceGroupNumber,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber,avg s valence electrons,avg p valence electrons,avg d valence electrons,avg f valence electrons,compound possible,max ionic char,avg ionic char,magmom_pa,bandgap,delta_e
0,In1,1,1.000000,1.000000,1.000000,1.000000,1.000000,49.0,49.0,0.0,49.000000,0.000000,49.0,75.0,75.0,0.0,75.000000,0.000000,75.0,114.818000,114.818000,0.000000,114.818000,0.000000,114.818000,429.75,429.75,0.00,429.750000,0.000000,429.75,13.0,13.0,0.0,13.000000,0.000000,13.0,5.0,5.0,0.0,...,5.0,5.0,0.0,5.000000,0.000000,5.0,24.260000,24.260000,0.0000,24.260000,0.000000,24.260000,0.000,0.000,0.000,0.000000,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,139.0,139.0,0.0,139.000000,0.000000,139.0,2.000000,1.000000,10.000000,0.0,True,0.000000,0.000000,2.700000e-05,0.0000,0.003319
1,Mg1,1,1.000000,1.000000,1.000000,1.000000,1.000000,12.0,12.0,0.0,12.000000,0.000000,12.0,68.0,68.0,0.0,68.000000,0.000000,68.0,24.305000,24.305000,0.000000,24.305000,0.000000,24.305000,923.00,923.00,0.00,923.000000,0.000000,923.00,2.0,2.0,0.0,2.000000,0.000000,2.0,3.0,3.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,22.890000,22.890000,0.0000,22.890000,0.000000,22.890000,0.000,0.000,0.000,0.000000,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,194.0,194.0,0.0,194.000000,0.000000,194.0,2.000000,0.000000,0.000000,0.0,True,0.000000,0.000000,-1.360000e-05,0.0000,0.039182
2,Be1,1,1.000000,1.000000,1.000000,1.000000,1.000000,4.0,4.0,0.0,4.000000,0.000000,4.0,67.0,67.0,0.0,67.000000,0.000000,67.0,9.012182,9.012182,0.000000,9.012182,0.000000,9.012182,1560.00,1560.00,0.00,1560.000000,0.000000,1560.00,2.0,2.0,0.0,2.000000,0.000000,2.0,2.0,2.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,7.890000,7.890000,0.0000,7.890000,0.000000,7.890000,0.000,0.000,0.000,0.000000,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,194.0,194.0,0.0,194.000000,0.000000,194.0,2.000000,0.000000,0.000000,0.0,True,0.000000,0.000000,1.000000e-07,0.0000,0.108143
3,Hf1,1,1.000000,1.000000,1.000000,1.000000,1.000000,72.0,72.0,0.0,72.000000,0.000000,72.0,45.0,45.0,0.0,45.000000,0.000000,45.0,178.490000,178.490000,0.000000,178.490000,0.000000,178.490000,2506.00,2506.00,0.00,2506.000000,0.000000,2506.00,4.0,4.0,0.0,4.000000,0.000000,4.0,6.0,6.0,0.0,...,8.0,8

In [16]:
X.rename(columns=lambda x: x.removeprefix('MagpieData '), inplace=True)
X

,comp,0-norm,2-norm,3-norm,5-norm,7-norm,10-norm,minimum Number,maximum Number,range Number,mean Number,avg_dev Number,mode Number,minimum MendeleevNumber,maximum MendeleevNumber,range MendeleevNumber,mean MendeleevNumber,avg_dev MendeleevNumber,mode MendeleevNumber,minimum AtomicWeight,maximum AtomicWeight,range AtomicWeight,mean AtomicWeight,avg_dev AtomicWeight,mode AtomicWeight,minimum MeltingT,maximum MeltingT,range MeltingT,mean MeltingT,avg_dev MeltingT,mode MeltingT,minimum Column,maximum Column,range Column,mean Column,avg_dev Column,mode Column,minimum Row,maximum Row,range Row,...,minimum NUnfilled,maximum NUnfilled,range NUnfilled,mean NUnfilled,avg_dev NUnfilled,mode NUnfilled,minimum GSvolume_pa,maximum GSvolume_pa,range GSvolume_pa,mean GSvolume_pa,avg_dev GSvolume_pa,mode GSvolume_pa,minimum GSbandgap,maximum GSbandgap,range GSbandgap,mean GSbandgap,avg_dev GSbandgap,mode GSbandgap,minimum GSmagmom,maximum GSmagmom,range GSmagmom,mean GSmagmom,avg_dev GSmagmom,mode GSmagmom,minimum SpaceGroupNumber,maximum SpaceGroupNumber,range SpaceGroupNumber,mean SpaceGroupNumber,avg_dev SpaceGroupNumber,mode SpaceGroupNumber,avg s valence electrons,avg p valence electrons,avg d valence electrons,avg f valence electrons,compound possible,max ionic char,avg ionic char,magmom_pa,bandgap,delta_e
0,In1,1,1.000000,1.000000,1.000000,1.000000,1.000000,49.0,49.0,0.0,49.000000,0.000000,49.0,75.0,75.0,0.0,75.000000,0.000000,75.0,114.818000,114.818000,0.000000,114.818000,0.000000,114.818000,429.75,429.75,0.00,429.750000,0.000000,429.75,13.0,13.0,0.0,13.000000,0.000000,13.0,5.0,5.0,0.0,...,5.0,5.0,0.0,5.000000,0.000000,5.0,24.260000,24.260000,0.0000,24.260000,0.000000,24.260000,0.000,0.000,0.000,0.000000,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,139.0,139.0,0.0,139.000000,0.000000,139.0,2.000000,1.000000,10.000000,0.0,True,0.000000,0.000000,2.700000e-05,0.0000,0.003319
1,Mg1,1,1.000000,1.000000,1.000000,1.000000,1.000000,12.0,12.0,0.0,12.000000,0.000000,12.0,68.0,68.0,0.0,68.000000,0.000000,68.0,24.305000,24.305000,0.000000,24.305000,0.000000,24.305000,923.00,923.00,0.00,923.000000,0.000000,923.00,2.0,2.0,0.0,2.000000,0.000000,2.0,3.0,3.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,22.890000,22.890000,0.0000,22.890000,0.000000,22.890000,0.000,0.000,0.000,0.000000,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,194.0,194.0,0.0,194.000000,0.000000,194.0,2.000000,0.000000,0.000000,0.0,True,0.000000,0.000000,-1.360000e-05,0.0000,0.039182
2,Be1,1,1.000000,1.000000,1.000000,1.000000,1.000000,4.0,4.0,0.0,4.000000,0.000000,4.0,67.0,67.0,0.0,67.000000,0.000000,67.0,9.012182,9.012182,0.000000,9.012182,0.000000,9.012182,1560.00,1560.00,0.00,1560.000000,0.000000,1560.00,2.0,2.0,0.0,2.000000,0.000000,2.0,2.0,2.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,7.890000,7.890000,0.0000,7.890000,0.000000,7.890000,0.000,0.000,0.000,0.000000,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,194.0,194.0,0.0,194.000000,0.000000,194.0,2.000000,0.000000,0.000000,0.0,True,0.000000,0.000000,1.000000e-07,0.0000,0.108143
3,Hf1,1,1.000000,1.000000,1.000000,1.000000,1.000000,72.0,72.0,0.0,72.000000,0.000000,72.0,45.0,45.0,0.0,45.000000,0.000000,45.0,178.490000,178.490000,0.000000,178.490000,0.000000,178.490000,2506.00,2506.00,0.00,2506.000000,0.000000,2506.00,4.0,4.0,0.0,4.000000,0.000000,4.0,6.0,6.0,0.0,...,8.0,8.0,0.0,8.000000,0.000000,8.0,22.200000,22.200000,0.0000,22.200000,0.000000,22.200000,0.000,0.000,0.000,0.000000,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,194.0,194.0,0.0,194.000000,0.000000,194.0,2.000000,0.000000,2.000000,14.0,True,0.000000,0.000000,-2.050000e-05,0.0000,0.071216
4,P1,1,1.000000,1.000000,1.000000,1.000000,1.000000,15.0,15.0,0.0,15.000000,0.000000,15.0,83.0,83.0,0.0,83.000000,0.000000,83.0,30.973762,30.973762,0.000000,30.973762,0.000000,30.973762,317.30,317.30,0.00,317.300000,0.000000,317.30,15.0,15.0,0.0,15.000000,0.000000,15.0,3.0,3.0,0.0,...,3.0,3.0,0.0,3.000000,0.000000,3.0,22.570238,22.570238,0.0000,22.570238,0.000000,22.570238,1.625,1.625,0.000,1.625000,0.000000,1.6

In [20]:
with open('feat_col_name.txt', 'r') as feat_file:
    feats = [x.strip() for x in feat_file.readlines()]

a = set(feats)
b = set(X.columns)

print(a.difference(b))
print(b.difference(a))

{'energy_pa', 'stability', 'volume_pa'}
{'delta_e', 'comp'}


In [21]:
X.to_csv('mpc_full_feats_raw.csv', index=False)

In [25]:
from sklearn.preprocessing import StandardScaler

exclude_cols = ['comp', 'delta_e']

num_X = X.drop(columns=exclude_cols)
scaler = StandardScaler()
scaler.fit(num_X)

/home/nicholas/anaconda3/envs/slices/lib/python3.9/site-packages/sklearn/utils/validation.py:615: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


StandardScaler()

In [35]:
def scaler_to_df(s: StandardScaler):
    return pd.DataFrame({
        'mean': s.mean_,
        'scale': s.scale_
    }, index=pd.Series(s.get_feature_names_out(), name='column'))

scale_df = scaler_to_df(scaler)
scale_df.to_csv('mpc_scales.csv')

In [38]:
X_scaled = X.copy()
X_scaled[num_X.columns] = scaler.transform(num_X)
X_scaled.describe()

/home/nicholas/anaconda3/envs/slices/lib/python3.9/site-packages/sklearn/utils/validation.py:615: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


,0-norm,2-norm,3-norm,5-norm,7-norm,10-norm,minimum Number,maximum Number,range Number,mean Number,avg_dev Number,mode Number,minimum MendeleevNumber,maximum MendeleevNumber,range MendeleevNumber,mean MendeleevNumber,avg_dev MendeleevNumber,mode MendeleevNumber,minimum AtomicWeight,maximum AtomicWeight,range AtomicWeight,mean AtomicWeight,avg_dev AtomicWeight,mode AtomicWeight,minimum MeltingT,maximum MeltingT,range MeltingT,mean MeltingT,avg_dev MeltingT,mode MeltingT,minimum Column,maximum Column,range Column,mean Column,avg_dev Column,mode Column,minimum Row,maximum Row,range Row,mean Row,...,minimum NUnfilled,maximum NUnfilled,range NUnfilled,mean NUnfilled,avg_dev NUnfilled,mode NUnfilled,minimum GSvolume_pa,maximum GSvolume_pa,range GSvolume_pa,mean GSvolume_pa,avg_dev GSvolume_pa,mode GSvolume_pa,minimum GSbandgap,maximum GSbandgap,range GSbandgap,mean GSbandgap,avg_dev GSbandgap,mode GSbandgap,minimum GSmagmom,maximum GSmagmom,range GSmagmom,mean GSmagmom,avg_dev GSmagmom,mode GSmagmom,minimum SpaceGroupNumber,maximum SpaceGroupNumber,range SpaceGroupNumber,mean SpaceGroupNumber,avg_dev SpaceGroupNumber,mode SpaceGroupNumber,avg s valence electrons,avg p valence electrons,avg d valence electrons,avg f valence electrons,compound possible,max ionic char,avg ionic char,magmom_pa,bandgap,delta_e
count,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398100e+04,8.398100e+04,8.398100e+04,8.398100e+04,8.398100e+04,8.398300e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,...,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398700e+04,8.398500e+04,8.398500e+04,8.398700e+04,8.398700e+04,83987.000000
mean,2.274089e-16,-1.410476e-15,6.172526e-16,-1.250410e-15,-4.033800e-16,3.011814e-16,-4.466960e-17,-7.309571e-17,1.790168e-16,-1.949219e-16,-2.436524e-17,-6.903484e-17,-8.121745e-18,2.977973e-16,-3.485582e-17,2.436524e-16,1.109972e-16,1.313015e-16,-9.610732e-17,3.248698e-17,-2.497437e-16,-2.707248e-17,8.663195e-17,-2.301161e-17,8.122326e-18,-5.414884e-18,-2.247177e-16,-4.007014e-16,-4.237147e-16,1.597353e-16,-2.842611e-17,2.328234e-16,-3.519423e-17,-2.829075e-16,3.898438e-16,-1.379851e-16,2.436524e-16,-2.734321e-16,-2.436524e-17,2.436524e-16,...,-6.497396e-17,2.165799e-17,-1.147197e-16,3.722467e-17,4.331598e-17,-1.380697e-16,1.759711e-17,2.165799e-16,-3.519423e-17,2.436524e-17,-9.644573e-18,1.705567e-16,1.082899e-17,1.191189e-16,-1.137044e-16,-7.038846e-17,-8.121745e-17,-4.331598e-17,1.133660e-17,3.434821e-17,-2.030436e-18,-4.873047e-17,-3.316379e-17,-5.752903e-18,-8.121745e-18,2.116730e-16,1.245334e-16,-3.248698e-17,-9.204645e-17,1.299479e-16,-3.614177e-16,-2.869683e-16,-2.355306e-16,8.121745e-17,-1.218262e-16,3.167556e-16,7.445111e-18,4.737685e-17,-1.001682e-16,-1.475312
std,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,...,1.000006e+

In [43]:
X_scaled.to_csv('mpc_full_feats_scaled.csv', index=False)